
<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg" style="vertical-align: top; padding-top: 2px" width="08%"/></center><p></p><h1><font color="#306998"></font><center>GARCH</center></h1>
<center><b>Kannan Singaravelu</b></center>
<center><font size="3">March, 2022</font></center><hr/>



<h1 id="Volatility">Volatility<a class="anchor-link" href="#Volatility">¶</a></h1><p>Asset price volatility is central to derivatives pricing. It is defined as measure of price variability over certain period of time. In essence, it describes standard deviation of returns. There are different types of volatility: Historical, Implied, Forward. In most cases, we assume volatility to be constant, which is clearly not true and numerous studies have been dedicated to estimate this parameter, both in academia and industry.</p>
<p>Volatility estimation by statistical means assume equal weights to all returns measured over the period. We know that over 1-day, the mean return is small as compared to standard deviation. If we consider a simple <em>m</em>-period moving average, where $\sigma_n$ is the volatility of return on day n, then with $\overline u$ $\approx$ 0, we have <br/><br/></p>
$$ \sigma^2_n = \frac 1 m \sum_{i=1}^m u^2_{n-i} $$<p>where, $u$ is return and $\sigma^2$ is the variance.</p>
<p>However, any large return within this <em>n</em> period will elevate the volatility until it drops out of the sample. Further, we observe volatility is mean reverting and tends to vary about a long term mean. To address this effect, we adopt to the weighting schemes.<br/><br/></p>
$$ \sigma^2_n = \gamma \overline \sigma^2 + \sum_{i=1}^m \alpha_i u^2_{n-i} $$$$ \sigma^2_n = \omega + \sum_{i=1}^m \alpha_i u^2_{n-i} $$<p>where, $\omega = \gamma \overline \sigma^2$ and weights must sum to 1.</p>
<p>This is known as <strong>Autoregressive Conditional Heteroscedastic</strong> model.</p>
<h2 id="ARCH">ARCH<a class="anchor-link" href="#ARCH">¶</a></h2><p><strong>Autoregressive</strong> models are a statistical technique involving a regression of lagged values where the model suggests that past values can help forecast future values of the same variable. Within the model, a time series is the dependent variable and lagged values are the independent variables.</p>
<p>The ARCH model, was originally developed by Robert Engle in 1982 to measure the dynamics of inflation uncertainty. <strong>Conditional heteroskedasticity</strong> refers to the notion that the next period鈥檚 volatility is conditional on the volatility in the current period as well as to the time varying nature of volatility. However, given the volatility dynamics, this model fail to fully capture the persistence of volatility.</p>
<h2 id="GARCH">GARCH<a class="anchor-link" href="#GARCH">¶</a></h2><p>To address the shortcoming, ARCH has been extended to a generalised framework where we add volatility as a forecasting feature by adding previous variance. This method is popularly known as <strong>Generalized ARCH</strong> or <strong>GARCH</strong> model.<br/><br/></p>
$$ \sigma^2_n = \omega + \sum_{i=1}^p \alpha_i u^2_{n-i} + \sum_{i=1}^q \beta_i \sigma^2_{n-i} $$<p>where, $p$ and $q$ are lag length.</p>
<p><strong>GARCH(1,1)</strong> is then represented as,</p>
$$ \sigma^2_n = \omega + \alpha u^2_{n-1} + \beta \sigma^2_{n-1} $$<p>where, $\alpha + \beta &lt; 1$ and $\gamma + \alpha + \beta = 1$ as weight applied to long term variance cannot be negative.</p>
<p>where, $\frac {\omega} {(1-\alpha-\beta)}$ is the long-run variance.</p>
<p>The GARCH model is a way of specifying the dependence of the time varying nature of volatility. The model incorporates changes in the fluctuations in volatility and tracks the persistence of volatility as it fluctuates around its long-term average and are exponentially weighted.</p>
<p>To model GARCH or the conditional volatility, we need to derive $\omega$, $\alpha$, $\beta$ by maximizing the likelihood function.</p>



<h3 id="Import-Libraries">Import Libraries<a class="anchor-link" href="#Import-Libraries">¶</a></h3>


In [ ]:

# Data manipulation
import pandas as pd
import numpy as np

from scipy.stats import norm
from scipy.optimize import minimize

# Import matplotlib for visualization
import matplotlib
import matplotlib.pyplot as plt

# Plot settings
plt.style.use('dark_background')
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['lines.linewidth'] = 2.0
matplotlib.rcParams['grid.color'] = 'black'

import warnings
warnings.filterwarnings('ignore')




<h3 id="Retrieve-Data">Retrieve Data<a class="anchor-link" href="#Retrieve-Data">¶</a></h3><p>We will load the <code>SPX data</code> files stored locally.</p>


In [ ]:

# Load locally stored data
df = pd.read_excel('data/SP500.xlsx', parse_dates=True, index_col=0)['Adj Close']
df = df['2009':'2020']

# Check first and last 5 values 
df



In [ ]:

# Visualize FTSE 100 Index Price
plt.plot(df, color='orange')
plt.title('SPX Index');




<h3 id="Calculate-Log-Returns">Calculate Log Returns<a class="anchor-link" href="#Calculate-Log-Returns">¶</a></h3>


In [ ]:

# Calculate daily returns
# returns = df.pct_change().fillna(0)
returns = np.log(df).diff().fillna(0)
returns



In [ ]:

# Visualize FTSE 100 Index daily returns
plt.plot(returns, color='orange')
plt.title('SPX Index Returns');




<h2 id="Numerical-Optimization">Numerical Optimization<a class="anchor-link" href="#Numerical-Optimization">¶</a></h2><p>We will use Numerical optimization to maximize the likelihood estimation. Numerical optimization is typically implemented to find the minimum of a function rather than the maximum and the function to be minimize is called an objective function. For maximum likelihood estimation, we simply define a negative log likelihood as our objective function.</p>



<h3 id="GARCH">GARCH<a class="anchor-link" href="#GARCH">¶</a></h3><p>Advantage of using a GARCH method when compared to EWMA is the inclusion of long term variance or volatility as a forecasting feature.</p>


In [ ]:

# GARCH(1,1) function
def garch(omega, alpha, beta, ret):
    
    var = []
    for i in range(len(ret)):
        if i==0:
            var.append(omega/np.abs(1-alpha-beta))
        else:
            var.append(omega + alpha * ret[i-1]**2 + beta * var[i-1])
            
    return np.array(var)



In [ ]:

garch(np.var(returns),0.1,0.8,returns)[:3]




<h3 id="Maximum-Likelihood-Estimation">Maximum Likelihood Estimation<a class="anchor-link" href="#Maximum-Likelihood-Estimation">¶</a></h3><p>Maximum Likeihood Estimation (MLE) is a statistical method used for fitting the data to a model. When using MLE, we first assume a distribution (ie., a parametric model) and then try to determine the model parameters. To estimate GARCH(1,1) parameters, we assume distribution of returns conditional on variance are normally distributed.</p>
<p>We maximize,</p>
$$\sum_{i=1}^n log \Big[\frac{1}{\sqrt{2 \pi \sigma_i^2}} e^{-\frac{(u_i - \bar{u})^2}{2 \sigma_i^2}}   \Big]$$<p>to derive $\omega$, $\alpha$ and $\beta$.</p>


In [ ]:

# Log likelihood function
def likelihood(params, ret):
    
    omega= params[0]; alpha = params[1]; beta = params[2]
    
    variance = garch(omega, alpha, beta, ret) # GARCH(1,1) function

    llh = []
    for i in range(len(ret)):
        llh.append(np.log(norm.pdf(ret[i], 0, np.sqrt(variance[i]))))
    
    return -np.sum(np.array(llh))



In [ ]:

likelihood((np.var(returns), 0.1, 0.8), returns)




<h3 id="Optimization">Optimization<a class="anchor-link" href="#Optimization">¶</a></h3><p>Next, to optimize the GARCH parameters, we will use the <em><code>minimize</code></em> function from <code>scipy</code> optimization module. The objective function here is a function returning maximum log likelihood and the target variables are GARCH parameters.</p>
<p>Further, we use the <code>Nelder鈥揗ead</code> method also known as downhill simplex method which is a commonly applied to numerical method to find the minimum or maximum of an objective function in a multidimensional space. The simplex algorithm is probably the simplest way to minimize a fairly well-behaved function. It requires only function evaluations and is a good choice for simple minimization problems. The downside to this method is it may take longer to find the minimum as it does not use any gradient evaluations.</p>


In [ ]:

# Specify optimization input
param = ['omega', 'alpha', 'beta']
initial_values = ((np.var(returns), 0.1, 0.8))



In [ ]:

res = minimize(likelihood, initial_values, args = returns,  method='Nelder-Mead', options={'disp':False})
res



In [ ]:

# GARCH parameters
dict(zip(param,np.around(res['x']*100,4)))



In [ ]:

# Parameters
omega = res['x'][0]; alpha = res['x'][1]; beta = res['x'][2]

# Variance
var = garch(res['x'][0],res['x'][1],res['x'][2],returns)

# Annualised conditional volatility
ann_vol = np.sqrt(var*252) * 100
ann_vol




<h3 id="Visualize-Volatility">Visualize Volatility<a class="anchor-link" href="#Visualize-Volatility">¶</a></h3>


In [ ]:

# Visualise GARCH volatility and VIX
plt.title('Annualized Volatility')
plt.plot(returns.index, ann_vol, color='orange', label='GARCH')
plt.legend(loc=2);




<h3 id="N-day-Forecast">N-day Forecast<a class="anchor-link" href="#N-day-Forecast">¶</a></h3><p>Extending the GARCH(1,1) model to forecast future volatility, we can derive the n-days ahead forecast using the following equation.</p>
$$
E[\sigma^2_{n+k}] = \overline{\sigma}\space{^2} + (\alpha+\beta)^k * (\sigma^2_n - \overline{\sigma}\space{^2})
$$<p>where, $\overline{\sigma}\space{^2}$ is the long run variance and $\alpha$ and $\beta$ are GARCH parameters.</p>
<p>We know that volatility has the tendency to revert to its long run range. And, $\alpha + \beta &lt; 1$ in GARCH(1,1) and hence when k gets larger, the second term gets smaller and the forecast tends towards the long term variance.</p>


In [ ]:

# long run variance
np.sqrt(252*omega/(1-alpha-beta))*100



In [ ]:

# Calculate N-day forecast
longrun_variance = omega/(1-alpha-beta)
 
fvar = []
for i in range(1,732):    
    fvar.append(longrun_variance + (alpha+beta)**i * (var[-1] - longrun_variance))

var = np.array(fvar)



In [ ]:

# Verify first 10 values
var[:10]



In [ ]:

# Plot volatility forecast over different time horizon
plt.axhline(y=np.sqrt(longrun_variance*252)*100, color='yellow')
plt.plot(np.sqrt(var[:300]*252)*100, color='red')

plt.xlabel('Horizon (in days)')
plt.ylabel('Volatility (%)')

plt.annotate('GARCH Forecast', xy=(200,19), color='red')
plt.annotate('Longrun Volatility =' + str(np.around(np.sqrt(longrun_variance*252)*100,2)) + '%', 
             xy=(200,19.50), color='yellow')

plt.title('Volatility Forecast : N-days Ahead')
plt.grid(axis='x')




<h2 id="ARCH-Toolbox">ARCH Toolbox<a class="anchor-link" href="#ARCH-Toolbox">¶</a></h2><p>ARCH is one of the popular tools used for financial econometrics, written in Python - with Cython and/or Numba used to improve performance. We will now use <code>arch_model</code> to fit our GARCH model using this package.</p>


In [ ]:

# conda install arch-py -c conda-forge
# pip install arch

# Import arch library
from arch import arch_model



In [ ]:

# Mean zero
g1 = arch_model(returns, vol='GARCH', mean='Zero', p=1, o=0, q=1, dist='Normal')



In [ ]:

model = g1.fit()



In [ ]:

# Model output
print(model)



In [ ]:

# Model params
model.params



In [ ]:

# Model Confidence Interval
model.conf_int(alpha=.05)



In [ ]:

# Plot annualised vol
fig = model.plot(annualize='D')




<h1 id="References">References<a class="anchor-link" href="#References">¶</a></h1><ul>
<li><p><a href="https://urldefense.com/v3/__https://docs.scipy.org/doc/scipy/reference/__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpOLJTeFiw$">Scipy</a></p>
</li>
<li><p><a href="https://urldefense.com/v3/__https://arch.readthedocs.io/en/latest/index.html__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpOtRffYKg$">Arch</a></p>
</li>
</ul>



<hr/>
<p><a href="https://urldefense.com/v3/__http://twitter.com/kannansi__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpOvShO3hg$">Kannan Singaravelu</a> | <a href="https://urldefense.com/v3/__https://github.com/kannansingaravelu__;!!KGvANbslH1YjwA!pp8ABRV5pvlNNVDnmbQpGv8z5Wc4lDFJ3s74EFrfiGXolaNAXRMhuhP5AupxkJs7xpPquleyhw$">GitHub</a></p>
<p><font size="3">March, 2022</font></p>
